In [ ]:
import logging
from pathlib import Path
from Pegasus.api import *
import glob
import os
import pandas as pd
logging.basicConfig(level = logging.DEBUG)

#Properties
props = Properties()
props["dagman.retry"] = "100"
props["pegasus.transfer.arguments"] = "-m 1"
props.write()


# import os
# os.environ['KAGGLE_USERNAME'] = "vedula"
# os.environ['KAGGLE_KEY'] = "482a5c14ced45f63f3698eacb8fa0c62"

# import kaggle
# kaggle.api.dataset_download_files('nikhilpandey360/chest-xray-masks-and-labels/download', path='.', unzip=True)

tc = TransformationCatalog()


preprocess = Transformation(
                "preprocess",
                site="local",
                pfn=str(Path(".").resolve() / "preprocess.py"),
                is_stageable=True
            )

data_split = Transformation(
                "data_split",
                site="local",
                pfn=str(Path(".").resolve() / "data_split.py"),
                is_stageable=True
            )

train_model = Transformation( 
                "train_model",
                site="local",
                pfn=str(Path(".").resolve() / "train_model.py"),
                is_stageable=True
            )

predict_masks = Transformation( 
                "predict_masks",
                site="local",
                pfn=str(Path(".").resolve() / "prediction.py"),
                is_stageable=True
            )


tc = TransformationCatalog().add_transformations(preprocess, data_split, train_model, predict_masks).write()

file_list = []
output_list = []

rc = ReplicaCatalog()

for file in glob.glob("./train_images/*.png"):
    f = file.replace("./train_images/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_images/").resolve() / f)
    
for file in glob.glob("./train_masks/*.png"):
    f = file.replace("./train_masks/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_masks/").resolve() / f)
    
for file in glob.glob("./test/*.png"):
    f = file.replace("./test/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./test/").resolve() / f)
    

    
checkpoint_file = "study_checkpoint.pkl"

if not os.path.isfile(checkpoint_file):
    df = pd.DataFrame(list())
    df.to_pickle(checkpoint_file)

rc.add_replica("local", checkpoint_file, Path(".").resolve() / checkpoint_file)
    
rc.write()

for filename in glob.glob("./train_images/*.png"):
    f = filename.replace("./train_images/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))


for filename in glob.glob("./train_masks/*.png"):
    f = filename.replace("./train_masks/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))

for filename in glob.glob("./test/*.png"):
    f = filename.replace("./test/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))

        
wf = Workflow("preprocess")
    
job_preprocess = Job(preprocess)\
                    .add_inputs(*file_list)\
                    .add_outputs(*output_list)

data_split_file = File("data_split.pkl")

job_data_split = Job(data_split)\
                    .add_inputs(*output_list)\
                    .add_outputs(data_split_file)

model = File("model.h5")

train_files = []

for filename in glob.glob("./train_images/*.png"):
    f = filename.replace("./train_images/", '').strip(".png")+"_norm.png"
    train_files.append(File(f))


for filename in glob.glob("./train_masks/*.png"):
    f = filename.replace("./train_masks/", '').strip(".png")+"_norm.png"
    train_files.append(File(f))

job_train = Job(train_model)\
                    .add_checkpoint(File(checkpoint_file), stage_out=True)\
                    .add_inputs(*train_files, data_split_file)\
                    .add_profiles(Namespace.PEGASUS, key="maxwalltime", value=1)\
                    .add_outputs(model)

test_list = []
mask_output = []

for filename in glob.glob("./test/*.png"):
    f = filename.replace("./test/", '').strip(".png")+"_norm.png"
    test_list.append(File(f))

for filename in glob.glob("./test/*.png"):
    f = filename.replace("./test/", '').strip(".png")+"_norm_mask.png"
    mask_output.append(File(f))

job_predict = Job(predict_masks)\
                    .add_inputs(model, *test_list, data_split_file)\
                    .add_outputs(*mask_output)


wf.add_jobs(job_preprocess, job_data_split, job_train, job_predict)

try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)  


################
# pegasus-plan #
################
2020.11.25 21:19:55.633 UTC:
2020.11.25 21:19:55.640 UTC:   -----------------------------------------------------------------------
2020.11.25 21:19:55.645 UTC:   File for submitting this DAG to HTCondor           : preprocess-0.dag.condor.sub
2020.11.25 21:19:55.650 UTC:   Log of DAGMan debugging messages                 : preprocess-0.dag.dagman.out
2020.11.25 21:19:55.656 UTC:   Log of HTCondor library output                     : preprocess-0.dag.lib.out
2020.11.25 21:19:55.662 UTC:   Log of HTCondor library error messages             : preprocess-0.dag.lib.err
2020.11.25 21:19:55.669 UTC:   Log of the life of condor_dagman itself          : preprocess-0.dag.dagman.log
2020.11.25 21:19:55.674 UTC:
2020.11.25 21:19:55.680 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with:
2020.11.25 21:19:55.690 UTC:   -----------------------------------------------------------------------
2020.11.25 21:19:56.970 UTC:  

[######################################------------]  76.0% ..Running (Completed: 19, Queued: 0, Running: 1, Failed: 0)